In [2]:
# 1) Instalar y cargar dataset Cats vs Dogs
# pip install tensorflow tensorflow-datasets -q

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

IMG_SIZE = 128
BATCH = 32
AUTOTUNE = tf.data.AUTOTUNE

(ds_train, ds_val), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:]'],
    with_info=True, as_supervised=True
)

def prep(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

train = ds_train.map(prep, num_parallel_calls=AUTOTUNE).shuffle(1000).batch(BATCH).prefetch(AUTOTUNE)
val   = ds_val.map(prep,   num_parallel_calls=AUTOTUNE).batch(BATCH).prefetch(AUTOTUNE)

# 2) Definir una CNN compacta
model = keras.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train, validation_data=val, epochs=3)  # sube epochs si tu GPU/CPU lo permite


Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 89.02 url/s] 


KeyError: "There is no item named 'PetImages\\\\Cat\\\\0.jpg' in the archive"

***Visualizar filtros de la primera capa***

In [ ]:
first_conv = model.layers[0]
weights, biases = first_conv.get_weights()  # shape: (3,3,3,32)
w = (weights - weights.min()) / (weights.max() - weights.min() + 1e-8)

n = min(8, w.shape[-1])
plt.figure(figsize=(12,2))
for i in range(n):
    f = w[:, :, :, i]
    # Colapsar a gris para visualizar (promedio de canales)
    f_gray = f.mean(axis=2)
    plt.subplot(1, n, i+1)
    plt.imshow(f_gray, cmap='gray')
    plt.axis('off')
    plt.title(f'Filtro {i}')
    plt.suptitle('Filtros de la primera capa')
plt.show()

***Visualizar activaciones (qué ve cada capa)***

In [ ]:
# Tomar un batch de validación y escoger 1 imagen
sample_batch = next(iter(val))
x_sample, y_sample = sample_batch
img = x_sample[0:1]  # (1, H, W, 3)

from tensorflow.keras import models
# Salidas de las primeras capas (Conv/Pool)
layers_to_show = []
for L in model.layers:
    if isinstance(L, (layers.Conv2D, layers.MaxPooling2D)):
        layers_to_show.append(L.output)

viz_model = models.Model(inputs=model.input, outputs=layers_to_show)
activations = viz_model.predict(img)

for layer, act in zip([l.name for l in model.layers if isinstance(l, (layers.Conv2D, layers.MaxPooling2D))], activations):
    num_filters = act.shape[-1]
    size = act.shape[1]
    tiles = min(8, num_filters)  # mostrar 8 canales
    display = np.zeros((size, size * tiles))
    for i in range(tiles):
        